In [56]:
%load_ext autoreload
%autoreload 2

from utils.hamiltonians import Max3satHamiltonian
from utils.qaoa import QAOA
from qiskit import transpile
from qiskit.circuit import QuantumCircuit
from utils.circuit_utils import calculate_expected_fidelity, calculate_expected_fidelity_ideal, calculate_swap_overhead
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, EstimatorV2 as Estimator
import numpy as np
import pandas as pd

hamiltonian = Max3satHamiltonian("./instances/uf20-01.cnf")

print(hamiltonian.formula.nv, hamiltonian.formula.clauses)
print(hamiltonian.get_pauli_list(), len(hamiltonian.get_pauli_list()))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
20 [[4, -18, 19], [3, 18, -5], [-5, -8, -15], [-20, 7, -16], [10, -13, -7], [-12, -9, 17], [17, 19, 5], [-16, 9, 15], [11, -5, -14], [18, -10, 13], [-3, 11, 12], [-6, -17, -8], [-18, 14, 1], [-19, -15, 10], [12, 18, -19], [-8, 4, 7], [-8, -9, 4], [7, 17, -15], [12, -7, -14], [-10, -11, 8], [2, -15, -11], [9, 6, 1], [-11, 20, -17], [9, -15, 13], [12, -7, -17], [-18, -2, 20], [20, 12, 4], [19, 11, 14], [-16, 18, -4], [-1, -17, -19], [-13, 15, 10], [-12, -14, -13], [12, -14, -7], [-7, 16, 10], [6, 10, 7], [20, 14, -16], [-19, 17, 11], [-7, 1, -20], [-5, 12, 15], [-4, -9, -13], [12, -11, -7], [-5, 19, -8], [1, 16, 17], [20, -14, -15], [13, -4, 10], [14, 7, 10], [-5, 9, 20], [10, 1, -19], [-16, -15, -1], [16, 3, -11], [-15, -10, 4], [4, -15, -3], [-10, -16, 11], [-8, 12, -5], [14, -6, 12], [1, 6, 11], [-13, -5, -1], [-7, -2, 12], [1, -20, 19], [-2, -13, -8], [15, 18, 4], [-11, 14, 9], [-6, -15, -2], [5, 

In [2]:
basis_gates = ["rx", "rz", "x", "y", "z", "h", "id", "cz"]
depth = 1
qaoa = QAOA(hamiltonian)
qaoa_circuit, cost_params, mixer_params = qaoa.naive_qaoa_circuit(depth)
bound_circuit = qaoa_circuit.assign_parameters({cost_params: [np.pi / 2.123 for param in cost_params], mixer_params: [np.pi / 3.123 for param in mixer_params]})
bound_circuit.measure_all()
transpiled_circuit = transpile(bound_circuit, basis_gates=basis_gates, optimization_level=3)

In [3]:
service = QiskitRuntimeService()

In [4]:
backend = service.backend("ibm_sherbrooke")

In [5]:
eagle_circuit = transpile(bound_circuit, backend=backend, optimization_level=3)

print(eagle_circuit.depth(), bound_circuit.depth())
print(eagle_circuit.count_ops(), bound_circuit.count_ops())

2585 191
OrderedDict([('rz', 4466), ('sx', 2911), ('ecr', 1281), ('x', 364), ('measure', 20), ('barrier', 1)]) OrderedDict([('cx', 295), ('rz', 231), ('rx', 20), ('measure', 20), ('barrier', 1)])


In [23]:
ost_operator = hamiltonian.get_sparse_pauli_operator()
observable = cost_operator.apply_layout(eagle_circuit.layout)
estimator = Estimator(backend=backend)

In [25]:
job = estimator.run([(eagle_circuit, observable)])

In [29]:
print(f"Job ID: {job.job_id()}")
print(f"Job Status: {job.status()}")

Job ID: crx7kt77wv80008fkzm0
Job Status: QUEUED


In [31]:
least_busy_backend = service.least_busy(operational=True, simulator=False)

print(least_busy_backend)

<IBMBackend('ibm_osaka')>


In [46]:
sampler = Sampler(backend=backend)
estimator = Estimator(backend=backend)
eagle_circuit = transpile(bound_circuit, backend=backend, optimization_level=3)
observable = cost_operator.apply_layout(eagle_circuit.layout)

print(eagle_circuit.depth(), bound_circuit.depth())
print(eagle_circuit.count_ops(), bound_circuit.count_ops())

2539 191
OrderedDict([('rz', 4546), ('sx', 2892), ('ecr', 1288), ('x', 405), ('measure', 20), ('barrier', 1)]) OrderedDict([('cx', 295), ('rz', 231), ('rx', 20), ('measure', 20), ('barrier', 1)])


In [49]:
sampler_job = sampler.run([eagle_circuit], shots=256)

In [43]:
job = estimator.run([(eagle_circuit, observable)])

/home/lax/Desktop/FPQA-MAX3-SAT/.venv/lib/python3.10/site-packages/qiskit_ibm_runtime/qiskit_runtime_service.py:879: UserWarning: Your current pending jobs are estimated to consume 623.1034379365619 quantum seconds, but you only have 600 quantum seconds left in your monthly quota; therefore, it is likely this job will be canceled
  warnings.warn(warning_message)


In [7]:
print(calculate_expected_fidelity(eagle_circuit, backend))
print(calculate_swap_overhead(transpiled_circuit, eagle_circuit))

(7.195686173181647e-12, 0.0001808796796848703, 152648674309, 6073)
328


In [68]:
def run_experiment(problem_name, backend, depth=1, iterations=10):
    hamiltonian = Max3satHamiltonian(problem_name)
    qaoa = QAOA(hamiltonian)
    qaoa_circuit, cost_params, mixer_params = qaoa.naive_qaoa_circuit(depth)
    fidelity_with_decoherence, fidelity, estimated_shots_with_decoherence, estimated_shots = 0.0, 0.0, 0, 0
    ideal_fidelity, ideal_estimated_shots = 0.0, 0
    num_two_qubit_gates, swap_overhead = 0, 0
    print(f">>> Parametrized circuit created...")
    for i in range(iterations):
        print(f">>> Compiling circuit: {problem_name}, Iteration: {i} ...") 
        params = {
            cost_params: np.random.uniform(low=0.0, high=2*np.pi, size=len(cost_params)),
            mixer_params: np.random.uniform(low=0, high=np.pi, size=len(mixer_params))
        }
        bound_circuit = qaoa_circuit.assign_parameters(params)
        bound_circuit.measure_all()
        transpiled_circuit = transpile(bound_circuit, backend=backend, optimization_level=3)
        fwd, f, eswd, es = calculate_expected_fidelity(transpiled_circuit, backend)
        ideal_f, ideal_es = calculate_expected_fidelity_ideal(transpiled_circuit)
        fidelity_with_decoherence += fwd
        fidelity += f
        estimated_shots_with_decoherence += eswd
        estimated_shots += es
        num_two_qubit_gates += transpiled_circuit.count_ops()["ecr"]
        swap_overhead += calculate_swap_overhead(bound_circuit, transpiled_circuit)
        ideal_fidelity += ideal_f
        ideal_estimated_shots += ideal_es
    avg_results = np.array([ideal_fidelity, ideal_estimated_shots, 
                            fidelity_with_decoherence, estimated_shots_with_decoherence, 
                            fidelity, estimated_shots, 
                            swap_overhead, num_two_qubit_gates], 
                           dtype=float) / iterations
    avg_results = avg_results.tolist()
    avg_results[-2] = int(np.ceil(avg_results[-2]))
    avg_results[-1] = int(np.ceil(avg_results[-1]))
    return avg_results

In [46]:
data_schema = [
    "Name", 
    "Depth",
    "Fidelity with Decoherence", 
    "Estimated Shots with Decoherence for 2/3 Success Rate", 
    "Fidelity without Decoherence", 
    "Estimated Shots without Decoherence for 2/3 Success Rate", 
    "Average Swap Overhead", 
    "Average Number of Two Qubit Gates", 
    "Iterations"
]

data = []

In [47]:
max_depth = 3
iterations = 10

for p in range(1, max_depth + 1):
    print(f">>> Compiling circuits with depth: {p} ...")
    result = run_experiment("./instances/uf20-01.cnf", backend, depth=p, iterations=iterations)
    data.append(["uf20-01.cnf", p] + result + [10])

>>> Compiling circuits with depth: 1 ...
>>> Compiling circuit: ./instances/uf20-01.cnf, Iteration: 0 ...
>>> Compiling circuit: ./instances/uf20-01.cnf, Iteration: 1 ...
>>> Compiling circuit: ./instances/uf20-01.cnf, Iteration: 2 ...
>>> Compiling circuit: ./instances/uf20-01.cnf, Iteration: 3 ...
>>> Compiling circuit: ./instances/uf20-01.cnf, Iteration: 4 ...
>>> Compiling circuit: ./instances/uf20-01.cnf, Iteration: 5 ...
>>> Compiling circuit: ./instances/uf20-01.cnf, Iteration: 6 ...
>>> Compiling circuit: ./instances/uf20-01.cnf, Iteration: 7 ...
>>> Compiling circuit: ./instances/uf20-01.cnf, Iteration: 8 ...
>>> Compiling circuit: ./instances/uf20-01.cnf, Iteration: 9 ...
>>> Compiling circuits with depth: 2 ...
>>> Compiling circuit: ./instances/uf20-01.cnf, Iteration: 0 ...


/home/ogz/Desktop/FPQA-MAX3-SAT/utils/circuit_utils.py:35: RuntimeWarning: divide by zero encountered in scalar divide
  estimated_shots = np.log(1 - DESIRED_SUCCESS_PROBABILITY) / np.log(1 - fidelity * decoherence_fidelity)


>>> Compiling circuit: ./instances/uf20-01.cnf, Iteration: 1 ...


/home/ogz/Desktop/FPQA-MAX3-SAT/utils/circuit_utils.py:35: RuntimeWarning: divide by zero encountered in scalar divide
  estimated_shots = np.log(1 - DESIRED_SUCCESS_PROBABILITY) / np.log(1 - fidelity * decoherence_fidelity)


>>> Compiling circuit: ./instances/uf20-01.cnf, Iteration: 2 ...


...

/home/ogz/Desktop/FPQA-MAX3-SAT/utils/circuit_utils.py:35: RuntimeWarning: divide by zero encountered in scalar divide
  estimated_shots = np.log(1 - DESIRED_SUCCESS_PROBABILITY) / np.log(1 - fidelity * decoherence_fidelity)


>>> Compiling circuit: ./instances/uf20-01.cnf, Iteration: 8 ...


/home/ogz/Desktop/FPQA-MAX3-SAT/utils/circuit_utils.py:35: RuntimeWarning: divide by zero encountered in scalar divide
  estimated_shots = np.log(1 - DESIRED_SUCCESS_PROBABILITY) / np.log(1 - fidelity * decoherence_fidelity)


>>> Compiling circuit: ./instances/uf20-01.cnf, Iteration: 9 ...


/home/ogz/Desktop/FPQA-MAX3-SAT/utils/circuit_utils.py:35: RuntimeWarning: divide by zero encountered in scalar divide
  estimated_shots = np.log(1 - DESIRED_SUCCESS_PROBABILITY) / np.log(1 - fidelity * decoherence_fidelity)


In [48]:
df_20 = pd.DataFrame(data, columns=data_schema)

df_20https://meet.google.com/shs-qpin-sds?authuser=0

,Name,Depth,Fidelity with Decoherence,Estimated Shots with Decoherence for 2/3 Success Rate,Fidelity without Decoherence,Estimated Shots without Decoherence for 2/3 Success Rate,Average Swap Overhead,Average Number of Two Qubit Gates,Iterations
0,uf20-01.cnf,1,7.249810e-12,4.170792e+11,1.468294e-04,8.406963e+03,328,1278,10
1,uf20-01.cnf,2,1.767152e-24,-inf,5.288556e-09,1.031245e+09,717,2741,10
2,uf20-01.cnf,3,1.474778e-37,-inf,9.188999e-14,1.054780e+14,1110,4216,10


In [69]:
data_schema_2 = [
    "Name", 
    "Depth",
    "Ideal Fidelity",
    "Ideal Estimated Shots",
    "Fidelity with Decoherence", 
    "Estimated Shots with Decoherence for 2/3 Success Rate", 
    "Fidelity without Decoherence", 
    "Estimated Shots without Decoherence for 2/3 Success Rate", 
    "Average Swap Overhead", 
    "Average Number of Two Qubit Gates", 
    "Iterations"
]

data_2 = []

In [70]:
max_depth = 1
iterations = 1

for p in range(1, max_depth + 1):
    print(f">>> Compiling circuits with depth: {p} ...")
    result = run_experiment("./instances/uuf50-01.cnf", backend, depth=p, iterations=iterations)
    data_2.append(["uuf50-01.cnf", p] + result + [10])

>>> Compiling circuits with depth: 1 ...
>>> Parametrized circuit created...
>>> Compiling circuit: ./instances/uuf50-01.cnf, Iteration: 0 ...


/home/ogz/Desktop/FPQA-MAX3-SAT/utils/circuit_utils.py:34: RuntimeWarning: divide by zero encountered in scalar divide
  estimated_shots_without_decoherence = np.log(1 - DESIRED_SUCCESS_PROBABILITY) / np.log(1 - fidelity)
/home/ogz/Desktop/FPQA-MAX3-SAT/utils/circuit_utils.py:35: RuntimeWarning: divide by zero encountered in scalar divide
  estimated_shots = np.log(1 - DESIRED_SUCCESS_PROBABILITY) / np.log(1 - fidelity * decoherence_fidelity)
/home/ogz/Desktop/FPQA-MAX3-SAT/utils/circuit_utils.py:48: RuntimeWarning: divide by zero encountered in scalar divide


In [72]:
max_depth = 1
iterations = 1

for p in range(1, max_depth + 1):
    print(f">>> Compiling circuits with depth: {p} ...")
    result = run_experiment("./instances/uuf75-01.cnf", backend, depth=p, iterations=iterations)
    data_2.append(["uuf75-01.cnf", p] + result + [10])

>>> Compiling circuits with depth: 1 ...
>>> Parametrized circuit created...
>>> Compiling circuit: ./instances/uuf75-01.cnf, Iteration: 0 ...


/home/ogz/Desktop/FPQA-MAX3-SAT/utils/circuit_utils.py:34: RuntimeWarning: divide by zero encountered in scalar divide
  estimated_shots_without_decoherence = np.log(1 - DESIRED_SUCCESS_PROBABILITY) / np.log(1 - fidelity)
/home/ogz/Desktop/FPQA-MAX3-SAT/utils/circuit_utils.py:35: RuntimeWarning: divide by zero encountered in scalar divide
  estimated_shots = np.log(1 - DESIRED_SUCCESS_PROBABILITY) / np.log(1 - fidelity * decoherence_fidelity)
/home/ogz/Desktop/FPQA-MAX3-SAT/utils/circuit_utils.py:48: RuntimeWarning: divide by zero encountered in scalar divide


In [74]:
max_depth = 1
iterations = 1

for p in range(1, max_depth + 1):
    print(f">>> Compiling circuits with depth: {p} ...")
    result = run_experiment("./instances/uuf100-01.cnf", backend, depth=p, iterations=iterations)
    data_2.append(["uuf100-01.cnf", p] + result + [10])

>>> Compiling circuits with depth: 1 ...
>>> Parametrized circuit created...
>>> Compiling circuit: ./instances/uuf100-01.cnf, Iteration: 0 ...


/home/ogz/Desktop/FPQA-MAX3-SAT/utils/circuit_utils.py:34: RuntimeWarning: divide by zero encountered in scalar divide
  estimated_shots_without_decoherence = np.log(1 - DESIRED_SUCCESS_PROBABILITY) / np.log(1 - fidelity)
/home/ogz/Desktop/FPQA-MAX3-SAT/utils/circuit_utils.py:35: RuntimeWarning: divide by zero encountered in scalar divide
  estimated_shots = np.log(1 - DESIRED_SUCCESS_PROBABILITY) / np.log(1 - fidelity * decoherence_fidelity)
/home/ogz/Desktop/FPQA-MAX3-SAT/utils/circuit_utils.py:48: RuntimeWarning: divide by zero encountered in scalar divide


In [76]:
max_depth = 3
iterations = 1

for p in range(1, max_depth + 1):
    print(f">>> Compiling circuits with depth: {p} ...")
    result = run_experiment("./instances/uf20-01.cnf", backend, depth=p, iterations=iterations)
    data_2.append(["uf20-01.cnf", p] + result + [10])

>>> Compiling circuits with depth: 1 ...
>>> Parametrized circuit created...
>>> Compiling circuit: ./instances/uf20-01.cnf, Iteration: 0 ...
>>> Compiling circuits with depth: 2 ...
>>> Parametrized circuit created...
>>> Compiling circuit: ./instances/uf20-01.cnf, Iteration: 0 ...


/home/ogz/Desktop/FPQA-MAX3-SAT/utils/circuit_utils.py:35: RuntimeWarning: divide by zero encountered in scalar divide
  estimated_shots = np.log(1 - DESIRED_SUCCESS_PROBABILITY) / np.log(1 - fidelity * decoherence_fidelity)


>>> Compiling circuits with depth: 3 ...
>>> Parametrized circuit created...
>>> Compiling circuit: ./instances/uf20-01.cnf, Iteration: 0 ...


/home/ogz/Desktop/FPQA-MAX3-SAT/utils/circuit_utils.py:35: RuntimeWarning: divide by zero encountered in scalar divide
  estimated_shots = np.log(1 - DESIRED_SUCCESS_PROBABILITY) / np.log(1 - fidelity * decoherence_fidelity)
/home/ogz/Desktop/FPQA-MAX3-SAT/utils/circuit_utils.py:48: RuntimeWarning: divide by zero encountered in scalar divide


In [78]:
max_depth = 1
iterations = 1

for p in range(1, max_depth + 1):
    print(f">>> Compiling circuits with depth: {p} ...")
    result = run_experiment("./instances/uuf125-01.cnf", backend, depth=p, iterations=iterations)
    data_2.append(["uuf125-01.cnf", p] + result + [10])

>>> Compiling circuits with depth: 1 ...
>>> Parametrized circuit created...
>>> Compiling circuit: ./instances/uuf125-01.cnf, Iteration: 0 ...


/home/ogz/Desktop/FPQA-MAX3-SAT/utils/circuit_utils.py:34: RuntimeWarning: divide by zero encountered in scalar divide
  estimated_shots_without_decoherence = np.log(1 - DESIRED_SUCCESS_PROBABILITY) / np.log(1 - fidelity)
/home/ogz/Desktop/FPQA-MAX3-SAT/utils/circuit_utils.py:35: RuntimeWarning: divide by zero encountered in scalar divide
  estimated_shots = np.log(1 - DESIRED_SUCCESS_PROBABILITY) / np.log(1 - fidelity * decoherence_fidelity)
/home/ogz/Desktop/FPQA-MAX3-SAT/utils/circuit_utils.py:48: RuntimeWarning: divide by zero encountered in scalar divide


In [81]:
df_50 = pd.DataFrame(data_2, columns=data_schema_2)

df_50.to_csv("superconducting_results.csv", index=False)

df_50

,Name,Depth,Ideal Fidelity,Ideal Estimated Shots,Fidelity with Decoherence,Estimated Shots with Decoherence for 2/3 Success Rate,Fidelity without Decoherence,Estimated Shots without Decoherence for 2/3 Success Rate,Average Swap Overhead,Average Number of Two Qubit Gates,Iterations
0,uuf50-01.cnf,1,1.414054e-31,-inf,0.000000e+00,-inf,0.000000e+00,-inf,1843,6393,10
1,uuf75-01.cnf,1,1.187888e-61,-inf,0.000000e+00,-inf,0.000000e+00,-inf,3796,12791,10
2,uuf100-01.cnf,1,6.109364e-100,-inf,0.000000e+00,-inf,0.000000e+00,-inf,6314,20871,10
3,uf20-01.cnf,1,7.029710e-07,1.562528e+06,1.543606e-11,7.115868e+10,2.201998e-04,4.987703e+03,329,1282,10
4,uf20-01.cnf,2,1.367137e-13,8.037058e+12,9.059694e-26,-inf,5.868762e-10,1.871625e+09,702,2698,10
5,uf20-01.cnf,3,5.896884e-21,-inf,1.048150e-38,-inf,3.182814e-14,3.447254e+13,1119,4242,10
6,uuf125-01.cnf,1,1.358350e-144,-inf,0.000000e+00,-inf,0.000000e+00,-inf,9254,30226,10


In [ ]:
max_depth = 3
iterations = 10
instances = ["uf20-01.cnf", "uuf50-01.cnf", "uuf75-01.cnf", "uuf100-01.cnf", "uuf125-01.cnf"]
data = []

for instance in instances:
    print(f"Running experiments with instance: {instance}")
    for p in range(1, max_depth + 1):
        print(f">>> Compiling circuits with depth: {p}...")
        result = run_experiment("./instances/" + instance, backend, depth=p, iterations=iterations)
        data.append([instance, p] + result + [iterations])